In [71]:
import os 
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import pandas as pd # type: ignore

In [72]:
from utils import get_dataset, get_model_by_tag, get_prompt_template
from LLMAnnotator import LLMAnnotator

In [73]:
os.environ['OPENAI_API_KEY_CLARIN']=os.getenv("OPENAI_API_KEY_CLARIN")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

token = os.getenv("OPENAI_API_KEY_CLARIN")

In [74]:
model_tag = 'gpt4omini'  
dataset_tag = '20_newsgroups'
prompt = 'no_samples'
temp = 0
number_of_experimet = 1

In [75]:
model = get_model_by_tag(model_tag, token, temp)  
dataset_for_annotation = get_dataset(dataset_tag=dataset_tag)
prompt_txt = get_prompt_template(dataset_tag, prompt)
examples_for_prompt = pd.DataFrame()

output_path = f'./results/{prompt}_temp{temp}/{model_tag}/{dataset_tag}_{model_tag}_{prompt}_temp{temp}_exp{number_of_experimet}.csv'
os.makedirs(f'./results/{prompt}_temp{temp}/{model_tag}', exist_ok=True)


In [76]:
df_annotated = pd.read_csv(output_path)

In [77]:
df_annotated = pd.read_csv(output_path)

In [78]:
not_nan_indices = df_annotated[~df_annotated['error'].isna()].index

print(not_nan_indices.tolist())  

[827]


In [79]:
df_annotated[~df_annotated['error'].isna()]

,text,output,logprobs,top_logprobs,original_label,error
827,henry spencer writes article fred j mccall wri...,NaN,NaN,NaN,NaN,Max retries reached: 3


In [80]:
dataset_for_annotation_subset = dataset_for_annotation.loc[not_nan_indices]
print(dataset_for_annotation_subset)


         label                                               text
827  sci.space  henry spencer writes article fred j mccall wri...


In [81]:
# print(model, dataset_for_annotation, prompt_txt, examples_for_prompt, output_path)

In [82]:
text_col = "text"
label_col = "label"
output_col = "output"

In [83]:
annotator = LLMAnnotator(
    model=model,
    dataset=dataset_for_annotation_subset,
    examples_for_prompt=examples_for_prompt,
    prompt_template=prompt_txt,
    column_text=text_col,
    column_label=label_col,
    column_output=output_col
)


In [84]:
df = annotator.get_results()

Nr: 0 Predicted label: sci.space


In [85]:
df

[{'text': 'henry spencer writes article fred j mccall writes given ive heard shuttle software rated level level many also keep mind achieved use sophisticated tools rather brute force ignorance attack problem challenger standdown simply threw hundreds people whole process hand think little inaccurate based feynmans account softwaredevelopment process standdown fred basically correct sophisticated tools lot effort painstaking care got one right challenger feynman cited software people exemplary compared engine people also noted software people starting feel management pressure cut corners hadnt give much yet among things software people worked hard get things right major preflight simulations considered failure simulations nearly bad inflight failure result number majorsimulation failures could counted one hand number inflight failures zero fred mentioned elsewhere applies flight software software runs experiments typically mostly put together experimenters gets nowhere near level tende

In [86]:
pd.DataFrame(df)

,text,output,logprobs,top_logprobs,original_label
0,henry spencer writes article fred j mccall wri...,sci.space,"{'content': [{'token': 's', 'bytes': [115], 'l...","[{'token': 's', 'bytes': [115], 'logprob': -4....",sci.space


In [87]:
df_new = pd.DataFrame(df)

if 'error' not in df_new.columns:
    df_annotated = df_annotated.drop('error', axis=1)
    df_annotated.iloc[not_nan_indices]=df_new.values  
    df_annotated.to_csv(output_path, index=False)

In [88]:
df_annotated.iloc[not_nan_indices]

,text,output,logprobs,top_logprobs,original_label
827,henry spencer writes article fred j mccall wri...,sci.space,"{'content': [{'token': 's', 'bytes': [115], 'l...","[{'token': 's', 'bytes': [115], 'logprob': -4....",sci.space


In [89]:
df_annotated

,text,output,logprobs,top_logprobs,original_label
0,article ashish arora writes excerpts netnewssc...,talk.politics.misc,"{'content': [{'token': 'talk', 'bytes': [116, ...","[{'token': 'talk', 'bytes': [116, 97, 108, 107...",talk.politics.misc
1,gateway telepath modem month actually one woul...,comp.os.ms-windows.misc,"{'content': [{'token': 'comp', 'bytes': [99, 1...","[{'token': 'comp', 'bytes': [99, 111, 109, 112...",comp.sys.ibm.pc.hardware
2,anybody provide advice concerning following tw...,sci.med,"{'content': [{'token': 's', 'bytes': [115], 'l...","[{'token': 's', 'bytes': [115], 'logprob': 0.0...",sci.med
3,article mike silverman writes anybody know goi...,rec.sport.baseball,"{'content': [{'token': 'rec', 'bytes': [114, 1...","[{'token': 'rec', 'bytes': [114, 101, 99], 'lo...",rec.sport.baseball
4,article stich christian e writes installed mot...,comp.sys.ibm.pc.hardware,"{'content': [{'token': 'comp', 'bytes': [99, 1...","[{'token': 'comp', 'bytes': [99, 111, 109, 112...",sci.electronics
...,...,...,...,...,...
1995,david sternlight writes article karl barrus wr...,talk.politics.misc,"{'content': [{'token': 'talk', 'bytes': [116, ...","[{'token': 'talk', 'bytes': [116, 97, 108, 107...",sci.crypt
1996,hello im looking information alphanumeric page...,sci.electronics,"{'content': [{'token': 's', 'bytes': [115], 'l...","[{'token': 's', 'bytes': [115], 'logprob': -1....",sci.electronics
1997,john r daker writes would like offocially nomi...,rec.motorcycles,"{'content': [{'token': 'rec', 'bytes': [114, 1...","[{'token': 'rec', 'bytes': [114, 101, 99], 'lo...",rec.motorcycles
1998,article writes looking information concerning ...,sci.space,"{'content': [{'token': 's', 'bytes': [115], 'l...","[{'token': 's', 'bytes': [115], 'logprob': 0.0...",sci.space


In [90]:
# annotator.save_results(output_path=output_path)

In [91]:
# df = pd.read_csv(output_path)
# df